In [ ]:
import torch
import torch.nn as nn

In [ ]:
def func(x):
    return torch.sin(x)


z = torch.randn(10, 2)
print(z)

# Fast method (copied from zuko repository)
with torch.enable_grad():
    z = z.clone().requires_grad_()
    x = func(z)
dj = torch.autograd.grad(x, z, torch.ones_like(x), create_graph=True)[0]
ladj = torch.log(torch.abs(dj))
ladj = torch.distributions.utils._sum_rightmost(ladj, -1)
print(ladj)

# Slow method
ladj = torch.zeros(z.shape[0])
for i in range(z.shape[0]):
    jac = torch.autograd.functional.jacobian(func, z[i, :], create_graph=True)
    ladj[i] = torch.log(torch.abs(torch.linalg.det(jac)))
print(ladj)